In [1]:
import numpy as np
import pandas as pd

c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\Dan\.conda\envs\iblenv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Load data
Align everything to the LEFT video

In [2]:
pid = '47be9ae4-290f-46ab-b047-952bc3a1a509'
VIDEO_DIR = 'D:/ibl_website_videos/proc'
DATA_DIR = './data'

In [5]:
# load timestamp files
body_ts = np.load(f'{DATA_DIR}/{pid}/{pid}_body_times_scaled.npy')
left_ts = np.load(f'{DATA_DIR}/{pid}/{pid}_left_times_scaled.npy')
right_ts = np.load(f'{DATA_DIR}/{pid}/{pid}_right_times_scaled.npy')

In [7]:
# Load csv files
dlc_right = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_right_dlc_scaled.csv')
dlc_left = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_left_dlc_scaled.csv')
dlc_body = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_body_dlc_scaled.csv')
trials = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_trials.csv',index_col=0)

In [8]:
# Load wheel
wheel = np.load(f'{DATA_DIR}/{pid}/{pid}_left_wheel_scaled.npy')

# Re-indexing
We're going to re-index everything to be matched to the *LEFT* video timestamps. So those timestamps will be in real-time but the body/right video will just tell you which frame corresponds to the left video frame.

(1) re-index videos, dlc, and wheel
(2) re-index trial events

In [24]:
def flatten(list_of_list):
  ret = []
  for list in list_of_list:
    ret += list
  return ret

In [49]:
# we will also be adding all the DLC points to this, we'll get them automatically and we'll get both the _x and _y coordinates for each
dlc_body_keys = ['tail_start']
dlc_left_keys = ['cl_nose_tip', 'cl_paw_l', 'cl_paw_r', 'cl_tube_top', 'cl_tongue_end_l', 'cl_tongue_end_r']
dlc_right_keys = ['cr_nose_tip', 'cr_paw_l', 'cr_paw_r', 'cr_tube_top', 'cr_tongue_end_l', 'cr_tongue_end_r']
# we'll handle pupil separately because it was cropped out
dlc_left_pupil_keys = ['pupil_right_r', 'pupil_left_r', 'pupil_top_r', 'pupil_bottom_r']

# generate the x/y versions
dlc_body_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_body_keys])
dlc_left_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_left_keys])
dlc_right_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_right_keys])
dlc_left_pupil_keys_xy = flatten([(x + '_x', x + '_y') for x in dlc_left_pupil_keys])

# 1 re-index videos, dlc, and wheel into one file
indexes = pd.DataFrame(columns=['left_ts','right_idx','body_idx','wheel'] + dlc_body_keys_xy + dlc_left_keys_xy + dlc_right_keys_xy + dlc_left_pupil_keys_xy)

In [50]:
print(indexes.keys())

Index(['left_ts', 'right_idx', 'body_idx', 'wheel', 'tail_start_x',
       'tail_start_y', 'cl_nose_tip_x', 'cl_nose_tip_y', 'cl_paw_l_x',
       'cl_paw_l_y', 'cl_paw_r_x', 'cl_paw_r_y', 'cl_tube_top_x',
       'cl_tube_top_y', 'cl_tongue_end_l_x', 'cl_tongue_end_l_y',
       'cl_tongue_end_r_x', 'cl_tongue_end_r_y', 'cr_nose_tip_x',
       'cr_nose_tip_y', 'cr_paw_l_x', 'cr_paw_l_y', 'cr_paw_r_x', 'cr_paw_r_y',
       'cr_tube_top_x', 'cr_tube_top_y', 'cr_tongue_end_l_x',
       'cr_tongue_end_l_y', 'cr_tongue_end_r_x', 'cr_tongue_end_r_y',
       'pupil_right_r_x', 'pupil_right_r_y', 'pupil_left_r_x',
       'pupil_left_r_y', 'pupil_top_r_x', 'pupil_top_r_y', 'pupil_bottom_r_x',
       'pupil_bottom_r_y'],
      dtype='object')


In [9]:
def get_dlc_coord(row, key):
  val = row[key]
  if np.isnan(val):
    val = -1
  return val

In [51]:
data_cols = {}

crop_data = pd.read_csv(f'{DATA_DIR}/{pid}/{pid}_left_pupil_rect.csv')

pupil_xy = [crop_data.x0_ss[0], crop_data.y0_ss[0]]

suffixes = ['_x', '_y']

for key in dlc_body_keys + dlc_left_keys + dlc_right_keys + dlc_left_pupil_keys:
  data_cols[key] = []

for i, lts in enumerate(left_ts):
  # get indexes
  right_idx = np.argmin(np.abs(right_ts-lts))
  body_idx = np.argmin(np.abs(body_ts-lts))

  row_data = [lts, right_idx, body_idx, wheel[i]]

  # pull the data columns
  for key in dlc_body_keys:
    for suffix in suffixes:
      row_data.append(get_dlc_coord(dlc_body.iloc[body_idx], key + suffix))
  for key in dlc_left_keys:
    for suffix in suffixes:
      row_data.append(get_dlc_coord(dlc_left.iloc[i], key[3:] + suffix))
  for key in dlc_right_keys:
    for suffix in suffixes:
      row_data.append(get_dlc_coord(dlc_right.iloc[right_idx], key[3:] + suffix))

  # get the pupil, and offset by the x0/y0 values
  for key in dlc_left_pupil_keys:
    for j, suffix in enumerate(suffixes):
      val = get_dlc_coord(dlc_left.iloc[i], key + suffix) - pupil_xy[j]
      row_data.append(val)
  
  indexes.loc[i] = row_data

In [52]:
indexes

,left_ts,right_idx,body_idx,wheel,tail_start_x,tail_start_y,cl_nose_tip_x,cl_nose_tip_y,cl_paw_l_x,cl_paw_l_y,...,cr_tongue_end_r_x,cr_tongue_end_r_y,pupil_right_r_x,pupil_right_r_y,pupil_left_r_x,pupil_left_r_y,pupil_top_r_x,pupil_top_r_y,pupil_bottom_r_x,pupil_bottom_r_y
0,128.551793,1.0,0.0,6.892360,265.811180,96.723560,235.769478,528.911655,486.421471,856.201782,...,437.022614,338.045647,88.537117,71.058556,45.266455,13.300587,85.470062,68.030132,66.463001,15.642567
1,128.585106,2.0,0.0,6.871689,265.811180,96.723560,240.165073,529.412323,486.421471,858.381119,...,429.043282,339.878632,88.647175,71.058556,65.671181,71.600094,85.520206,68.033310,79.751839,72.141155
2,128.618322,3.0,0.0,6.879904,265.811180,96.723560,240.843491,529.412323,483.691559,857.335281,...,429.808319,339.607361,88.647175,71.049908,65.881565,71.600094,85.520206,68.033310,79.751839,72.135864
3,128.651634,4.0,1.0,6.906515,265.811180,97.347805,240.912560,525.813385,486.212540,858.381119,...,429.772568,339.912926,88.707546,70.941601,65.881565,71.689255,85.538040,67.967258,79.832333,72.135723
4,128.684946,4.0,2.0,6.936661,266.493961,97.537090,241.422619,525.813385,489.314079,855.914459,...,429.772568,339.912926,88.766247,70.921886,65.856670,71.806091,85.593018,67.913799,80.020905,72.100376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108807,3751.422836,90819.0,109487.0,-68.219765,264.434704,103.021736,241.840805,527.859661,386.796989,897.531815,...,434.440804,341.919983,88.271889,70.641102,65.128208,72.301445,85.422363,68.046959,85.641724,73.302864
108808,3751.456148,90820.0,109488.0,-68.219704,264.473927,102.944969,241.840805,527.434341,386.796989,897.531815,...,434.440804,341.919983,88.239120,70.641102,65.103878,72.176102,85.404922,68.059074,85.600967,73.302864
108809,3751.489364,90821.0,109489.0,-68.219642,265.099899,103.549133,242.366066,526.252388,386.800995,897.563934,...,434.440865,341.919991,88.239120,70.744862,65.103642,72.176102,85.393147,68.159580,85.596954,73.400421
108810,3751.522676,90822.0,109490.0,-68.219581,264.473927,102.944969,243.270241,526.372253,386.800995,897.563934,...,469.543972,289.098434,88.234318,70.747437,65.058990,72.198067,85.388828,68.167702,85.575714,73.409920


In [53]:
for key in indexes.keys():
    dat = indexes[key].values.astype(np.float32)
    with open(f'{DATA_DIR}/{pid}/{pid}.{key}.bytes', 'wb') as file:
        file.write(dat.tobytes())

In [51]:
trials

,start_timestamp,stim_on_timestamp,feedback_timestamp,stimulus_side,contrast,feedback
0,31.277572,45.118421,45.700196,R,1.000,1.0
1,47.886711,48.453154,48.782721,L,0.250,1.0
2,50.954937,51.502883,51.860947,R,0.250,1.0
3,54.061793,56.286237,56.915608,R,0.125,1.0
4,59.090824,61.585941,61.897343,R,0.250,1.0
...,...,...,...,...,...,...
419,2516.027339,2516.627579,2516.716403,L,0.250,-1.0
420,2520.360605,2520.992409,2529.540654,L,0.125,-1.0
421,2533.177524,2534.674707,2537.883253,L,0.125,-1.0
422,2541.491992,2542.175924,2548.352839,L,1.000,-1.0


In [43]:
# 2 re-index the trial events
for i, row in trials.iterrows():
    # replace the timestamps with the index
    start_idx = np.argmin(np.abs(row['start_timestamp']-left_ts))
    stim_on_idx = np.argmin(np.abs(row['stim_on_timestamp']-left_ts))
    feedback_idx = np.argmin(np.abs(row['feedback_timestamp']-left_ts))
    trials.loc[i,'start_timestamp'] = start_idx
    trials.loc[i,'stim_on_timestamp'] = stim_on_idx
    trials.loc[i,'feedback_timestamp'] = feedback_idx

In [44]:
trials

,start_timestamp,stim_on_timestamp,feedback_timestamp,stimulus_side,contrast,feedback
0,0.0,90.0,88.0,L,1.0000,-1.0
1,177.0,215.0,228.0,R,0.1250,1.0
2,288.0,320.0,328.0,L,0.1250,1.0
3,389.0,429.0,442.0,L,1.0000,1.0
4,502.0,545.0,596.0,L,0.2500,1.0
...,...,...,...,...,...,...
420,73038.0,73052.0,74854.0,R,0.0625,-1.0
421,74958.0,74960.0,0.0,L,0.1250,1.0
422,76560.0,0.0,0.0,R,0.0625,-1.0
423,76799.0,0.0,0.0,R,0.2500,-1.0


In [46]:
trials.to_csv(f'{DATA_DIR}/{pid}/{pid}.trials.csv', index = False)